# 01 – Exploração do Snapshot

Primeiro notebook do **creditlab**.

Nele, vamos:
1. Instanciar o `CreditDataSynthesizer`.
2. Gerar a *safra 0* (snapshot) para alguns grupos homogêneos.
3. Fazer uma exploração inicial: contagem de contratos por grupo, estatísticas descritivas e um histograma de dias em atraso.

⚠️ Este notebook assume que o módulo `credit_data_synthesizer.py` está no mesmo diretório ou em `PYTHONPATH`.

In [ ]:
import sys
from pathlib import Path

# Adiciona o diretório-pai ao sys.path
sys.path.append(str(Path().resolve().parent))

import pandas as pd
import matplotlib.pyplot as plt


from importlib import reload
import credit_data_synthesizer as cds
reload(cds)  # garante versão atual no notebook

from credit_data_synthesizer import (
    CreditDataSynthesizer,
    default_group_profiles,   # função helper já incluída no módulo
)

## Parâmetros iniciais

In [ ]:
# ----- parâmetros que você quer -----
n_groups = 10
contracts_per_group = 10_000
n_safras = 36
seed = 0

# 1) Obtenha os perfis padrão e pegue só os três primeiros
group_profiles = default_group_profiles(n_groups)

# 2) Instancie o sintetizador
synth = CreditDataSynthesizer(
    group_profiles=group_profiles,
    contracts_per_group=contracts_per_group,
    n_safras=n_safras,
    random_seed=24,
)

In [5]:
df_panel['safra'].value_counts()

safra
202507    100000
202508    100000
202509    100000
202510    100000
202511    100000
202512    100000
202601    100000
202602    100000
202603    100000
202604    100000
202605    100000
202606    100000
202607    100000
202608    100000
202609    100000
202610    100000
202611    100000
202612    100000
202701    100000
202702    100000
202703    100000
202704    100000
202705    100000
202706    100000
202707    100000
202708    100000
202709    100000
202710    100000
202711    100000
202712    100000
202801    100000
202802    100000
202803    100000
202804    100000
202805    100000
202806    100000
Name: count, dtype: int64

## Geração da safra 0

In [ ]:
# 3) Gere os dados
df_snapshot, df_panel, df_trace = synth.generate()

print(f"Snapshot gerado com {len(df_snapshot):,} linhas")
print(f"Snapshot gerado com {len(df_panel):,} linhas")
print(f"Snapshot gerado com {len(df_trace):,} linhas")
display(df_snapshot.head())

Snapshot gerado com 100,000 linhas
Snapshot gerado com 3,600,000 linhas
Snapshot gerado com 18,583 linhas


### Contagem de contratos por `grupo_homogeneo`

In [ ]:
df_snapshot['grupo_homogeneo'].value_counts()

### Estatísticas descritivas das variáveis numéricas

In [ ]:
cols_numericas = df_snapshot.select_dtypes(include='number').columns.drop(['id_contrato', 'id_cliente'])

agg_df = (
    df_snapshot[cols_numericas.tolist() + ['grupo_homogeneo']]
    .groupby('grupo_homogeneo')
    .agg(['min', 'mean', 'max'])
)

# Flatten columns
agg_df.columns = ['_'.join(col) for col in agg_df.columns]
agg_df.reset_index(inplace=True)

# Supondo que agg_df tenha a coluna 'grupo_homogeneo'
agg_df = agg_df.copy()
agg_df['grupo_idx'] = agg_df['grupo_homogeneo'].str.extract(r'GH(\d+)').astype(int)
agg_df = agg_df.sort_values('grupo_idx').drop(columns='grupo_idx')

agg_df.head(10)

### Distribuição de `dias_atraso`

In [ ]:
plt.figure()
df_snapshot['dias_atraso'].hist(bins=30)
plt.xlabel('Dias em atraso')
plt.ylabel('Número de contratos')
plt.title('Histograma de dias em atraso – safra 0')
plt.show()

In [ ]:
df_snapshot.shape